In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import plotly.figure_factory as ff


In [239]:
# TimeInterval
# A TimeInterval is a tuple of (start, end)
class TimeInterval():
    """TimeInterval class, define a time interval
    start and end are datetime object
    TimeInterval can be combined by using the + operator if intervals overlap, they will be combined into one interval
    TimeInterval can be subtracted by using the - operator, the result will be the availability of the first object minus the availability of the second object
    """

    def __init__(self, start, end):
        self.start = start
        self.end = end

    # Combine two time intervals if they overlap, return Intervals if they do not overlap
    def __add__(self, other):
        if self.start > other.end or self.end < other.start:
            return TimeIntervals([self, other])
        else:
            return TimeIntervals([TimeInterval(min(self.start, other.start), max(self.end, other.end))])

    def __sub__(self, other):
        if self.start >= other.end or self.end <= other.start:
            return TimeIntervals([self])
        elif self.start < other.start and self.end > other.end:
            return TimeIntervals([TimeInterval(self.start, other.start), TimeInterval(other.end, self.end)])
        elif self.start < other.start and self.end <= other.end:
            return TimeIntervals([TimeInterval(self.start, other.start)])
        elif self.start >= other.start and self.end > other.end:
            return TimeIntervals([TimeInterval(other.end, self.end)])

    def __str__(self):
        return '({0}, {1})'.format(self.start, self.end)

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.start == other.start and self.end == other.end

    def __hash__(self):
        return hash(self.__str__())
    
    def __lt__(self, other):
        return self.start < other.start

    def __gt__(self, other):
        return self.start > other.start
    
    def __le__(self, other):
        return self.start <= other.start

    def __ge__(self, other):
        return self.start >= other.start




# TimeIntervals
# A TimeIntervals is a list of TimeInterval
class TimeIntervals():
    """TimeIntervals class, define a list of time intervals
    list of datetime interval, each interval is a tuple of (start, end)
    start and end are datetime object
    TimeIntervals can be combined by using the + operator if intervals overlap, they will be combined into one interval
    TimeIntervals can be subtracted by using the - operator, the result will be the availability of the first object minus the availability of the second object
    """

    def __init__(self, intervals):
        self.intervals = intervals

    def __add__(self, other):
        intervals = self.intervals + other.intervals
        intervals.sort(key=lambda x: x.start)
        new_intervals = [intervals[0]]
        for i in range(1, len(intervals)):
            if intervals[i].start <= new_intervals[-1].end:
                new_intervals[-1] = TimeInterval(new_intervals[-1].start, max(intervals[i].end, new_intervals[-1].end))
            else:
                new_intervals.append(intervals[i])
        return TimeIntervals(new_intervals)

    def __sub__(self, other):
        intervals = self.intervals
        for interval in other.intervals:
            new_intervals = []
            for i in range(len(intervals)):
                if intervals[i].end <= interval.start or intervals[i].start >= interval.end:
                    new_intervals.append(intervals[i])
                elif intervals[i].start < interval.start and intervals[i].end > interval.end:
                    new_intervals.append(TimeInterval(intervals[i].start, interval.start))
                    new_intervals.append(TimeInterval(interval.end, intervals[i].end))
                elif intervals[i].start < interval.start and intervals[i].end <= interval.end:
                    new_intervals.append(TimeInterval(intervals[i].start, interval.start))
                elif intervals[i].start >= interval.start and intervals[i].end > interval.end:
                    new_intervals.append(TimeInterval(interval.end, intervals[i].end))
            intervals = new_intervals
        return TimeIntervals(intervals)

    def get_hours(self):
        hours = 0
        for interval in self.intervals:
            hours += (interval.end - interval.start).total_seconds() / 3600
        return hours

    # Plot the availability
    def plot(self):
        # create dict for plotly
        df = []
        for interval in self.intervals:
            df.append(dict(Task='Time', Start=interval.start, Finish=interval.end))
        fig = ff.create_gantt(df, colors=['rgb(255, 0, 0)'], index_col='Task', show_colorbar=True, group_tasks=True)
        fig.show()

    def combine(self):
        intervals = self.intervals
        intervals.sort(key=lambda x: x.start)
        new_intervals = [intervals[0]]
        for i in range(1, len(intervals)):
            if intervals[i].start <= new_intervals[-1].end:
                new_intervals[-1] = TimeInterval(new_intervals[-1].start, max(intervals[i].end, new_intervals[-1].end))
            else:
                new_intervals.append(intervals[i])
        return TimeIntervals(new_intervals)

In [245]:
# Example

t1 = TimeInterval(datetime(2020, 1, 1, 0, 0), datetime(2020, 1, 1, 1, 0))
t2 = TimeInterval(datetime(2020, 1, 1, 2, 0), datetime(2020, 1, 1, 3, 0))
t3 = TimeInterval(datetime(2020, 1, 1, 4, 0), datetime(2020, 1, 1, 5, 0))
t4 = TimeInterval(datetime(2020, 1, 1, 6, 0), datetime(2020, 1, 1, 7, 0))
t5 = TimeInterval(datetime(2020, 1, 1, 0, 30), datetime(2020, 1, 1, 1, 30))
t6 = TimeInterval(datetime(2020, 1, 1, 2, 30), datetime(2020, 1, 1, 3, 30))
t7 = TimeInterval(datetime(2020, 1, 1, 4, 30), datetime(2020, 1, 1, 5, 30))
t8 = TimeInterval(datetime(2020, 1, 1, 6, 30), datetime(2020, 1, 1, 7, 30))

ts1 = TimeIntervals([t1, t2, t3, t4]).combine()
ts2 = TimeIntervals([t5, t6, t7, t8]).combine()
(ts1 + ts2).plot()
(ts1 - ts2).plot()
(ts2 - ts1).plot()



